<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/Quantum_Finance_OptimizationPortfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance qiskit
!pip install qiskit-aqua
!pip uninstall qiskit
!pip install qiskit
!pip install --upgrade qiskit


import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import numpy as np
import pandas_datareader as pdr
from datetime import datetime, timedelta
from qiskit import Aer, execute, QuantumCircuit, QuantumRegister
#from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua import QuantumInstance
from qiskit.providers.ibmq import BaseBackend

# Define the portfolio symbols and time frame
symbol = "AAPL"
start = "2020-01-01"
end = "2022-03-24"

import pandas_datareader as pdr
import datetime

# Define the stock symbol and time frame
symbol = "AAPL"
start = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2022, 1, 1)

# Download historical data from Yahoo Finance
#prices = pdr.DataReader(symbol, "yahoo", start, end)["Adj Close"]


#print(prices)

#################################

# Replace YOUR_API_KEY with your actual Alpha Vantage API key
api_key = "071JPJ0PHQP75TC2"

# Define the stock symbol and time frame
symbol = "AAPL"
start = pd.to_datetime("2020-01-01")
end = pd.to_datetime("2022-01-01")

# Download historical data from Alpha Vantage
ts = TimeSeries(key=api_key, output_format="pandas")
prices, _ = ts.get_daily_adjusted(symbol=symbol, outputsize="full")
prices = prices['5. adjusted close'].to_frame()

print(prices.columns)
print(prices.head())

# Define the objective function for the portfolio optimization
def objective_function(weights):
    returns = np.dot(weights, prices.pct_change().mean())
    risks = np.sqrt(np.dot(weights.T, np.dot(prices.pct_change().cov(), weights)))
    return returns / risks

# Set up the quantum circuit
n = len(prices.columns)
print(prices.columns)
q = QuantumRegister(n)
qc = QuantumCircuit(q)

# Define the oracle for the portfolio optimization
def portfolio_oracle(weights, q, qc):
    global n
    for i in range(n):
        qc.ry(2 * np.arcsin(np.sqrt(weights[i])), q[i])
    
    qc.cz(q[0], q[1])
    qc.cz(q[1], q[2])
    qc.cz(q[0], q[2])
    qc.cz(q[2], q[1])
    qc.cz(q[0], q[1])
    for i in range(n):
        qc.ry(-2 * np.arcsin(np.sqrt(weights[i])), q[i])



# Define the Grover iteration
def grover_iteration(q, qc, oracle):
    qc.h(q)
    oracle(qc)
    qc.h(q)
    for i in range(n):
        qc.z(q[i])
    qc.cz(q[0], q[1])
    qc.cz(q[1], q[2])
    qc.cz(q[0], q[2])
    qc.cz(q[2], q[1])
    qc.cz(q[0], q[1])
    for i in range(n):
        qc.z(q[i])
    qc.h(q)

# Run the portfolio optimization using the quantum circuit
optimizer = COBYLA()
result = optimizer.optimize(n, objective_function, initial_point=np.ones(n) / n)
weights = result[0]
qc = QuantumCircuit(q)
portfolio_oracle(weights, q, qc)
for i in range(5):
    grover_iteration(q, qc, lambda qc: portfolio_oracle(weights, q, qc))
result = execute(qc, Aer.get_backend("qasm_simulator")).result()
counts = result.get_counts()
optimal_weights = np.zeros(n)
for state, count in counts.items():
  optimal_weights[int(state[::-1], 2)] = count
  optimal_weights /= sum(optimal_weights)

print("Optimized weights:", optimal_weights)
print("Expected return:", np.dot(optimal_weights, prices.pct_change().mean()))
print("Expected risk:", np.sqrt(np.dot(optimal_weights.T, np.dot(prices.pct_change().cov(), optimal_weights))))






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: qiskit-terra
    Found existing installation: qiskit-terra 0.23.3
    Uninstalling qiskit-terra-0.23.3:
      Successfully uninstalled qiskit-terra-0.23.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qiskit 0.42.1 requires qiskit-terra==0.23.3, but you have qiskit-terra 0.17.4 which is incompatible.
qiskit-ibmq-provider 0.20.2 requires qiskit-terra>=0.18.0, but you have qiskit-terra 0.17.4 which is incompatible.
qiskit-aer 0.12.0 requires qiskit-terra>=0.21.0, but you have qiskit-terra 0.17.4 which is incompatible.
Found existing installation: qiskit 0.42.1
Unins